For now, this benchmark file simply loads our existing full benchmark results (265 MB) from [ml-evs/modnet-matbench](https://github.com/ml-evs/modnet-matbench) and exports them in the matbench format. Code for featurisation, hyperparameter optimisation and the final predictions themselves can be found in the aforementioned repository.

In [1]:
def download_and_extract(url, fname):
    import urllib
    from zipfile import ZipFile
    import os
    if os.path.exists(fname):
        print(f"File {fname} already found, will not redownload.")
        return
    
    response = urllib.request.urlretrieve(url, fname)
    with ZipFile(fname, "r") as _zip:
        _zip.extractall(".")
        
fname = "modnet-matbench-0.2"
# url = f"https://github.com/ml-evs/modnet-matbench/archive/refs/tags/v0.1.zip"
url = "https://github.com/ml-evs/modnet-matbench/archive/refs/tags/v0.2.zip"
download_and_extract(url, fname + ".zip")

In [2]:
import pickle
import numpy as np
from matbench.bench import MatbenchBenchmark

mb = MatbenchBenchmark(
    autoload=False, 
    subset=[
        'matbench_dielectric', 
        'matbench_jdft2d', 
        'matbench_steels', 
        'matbench_expt_gap', 
        'matbench_phonons',
        # The two elastic datasets still currently contain huge outliers
        'matbench_log_gvrh',
        'matbench_log_kvrh',
        # Cannot currently assess classification task accuracy correctly...
        # 'matbench_glass', 
        # 'matbench_expt_is_metal', 
    ],
)

results_locs = {task.dataset_name: f"{fname}/{task.dataset_name}/results/{task.dataset_name}_results.pkl" for task in mb.tasks}
results_locs["matbench_log_gvrh"] = results_locs["matbench_log_kvrh"] = f"{fname}/matbench_elastic/results/matbench_elastic_results.pkl"
target_key_map = {"matbench_log_gvrh": "log10G_VRH", "matbench_log_kvrh": "log10K_VRH"}

2021-06-10 16:14:49 INFO     Initialized benchmark 'matbench_v0.1' with 7 tasks: 
['matbench_dielectric',
 'matbench_jdft2d',
 'matbench_steels',
 'matbench_expt_gap',
 'matbench_phonons',
 'matbench_log_gvrh',
 'matbench_log_kvrh']


In [3]:
for task in mb.tasks:
    task.load()
    
    with open(results_locs[task.dataset_name], "rb") as f:
        results = pickle.load(f)
        
    for fold_ind, fold in enumerate(task.folds):
        if task.dataset_name in target_key_map:
            predictions = results["predictions"][fold_ind][target_key_map[task.dataset_name]].values.flatten()
        else:
            predictions = results["predictions"][fold_ind].values.flatten()
            
        if task.dataset_name in ("matbench_glass", "matbench_expt_is_metal"):
            predictions_ = predictions[:, 1]
            
        task.record(fold, predictions)
        
    task.df = None
    

2021-06-10 16:14:52 INFO     Loading dataset 'matbench_dielectric'...
2021-06-10 16:14:58 INFO     Dataset 'matbench_dielectric loaded.
2021-06-10 16:14:58 INFO     Recorded fold matbench_dielectric-0 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_dielectric-1 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_dielectric-2 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_dielectric-3 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_dielectric-4 successfully.
2021-06-10 16:14:58 INFO     Loading dataset 'matbench_jdft2d'...
2021-06-10 16:14:58 INFO     Dataset 'matbench_jdft2d loaded.
2021-06-10 16:14:58 INFO     Recorded fold matbench_jdft2d-0 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_jdft2d-1 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_jdft2d-2 successfully.
2021-06-10 16:14:58 INFO     Recorded fold matbench_jdft2d-3 successfully.
2021-06-10 16:14:58 INFO     Recorded fol

In [4]:
mb.to_file("results.json.gz")

2021-06-10 16:17:30 INFO     Successfully wrote MatbenchBenchmark to file 'results.json.gz'.
